In [1]:
# importing necessary libraries
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import numpy as np

In [2]:
# Initialize a counter and an empty DataFrame for storing final results
movie_counter = 0
final_data = pd.DataFrame()

In [3]:
# give number of page were start scraping 
starting_page = 1
# give number of page were end scraping 
ending_page = 2

In [4]:
# Loop through the movie listing pages (pagination)
for page_number in range(starting_page,(ending_page + 1)):
    # Request the movie listing webpage
    listing_page_content = requests.get(f"https://themoviezflix.cn.com/page/{page_number}/").text
    listing_page_soup = BeautifulSoup(listing_page_content, "lxml")

    # Loop through all movie links on the page
    for movie_link_tag in listing_page_soup.find_all("a", class_="post-image"):
        movie_url = movie_link_tag.get("href")
        movie_page_content = requests.get(str(movie_url)).text
        movie_page_soup = BeautifulSoup(movie_page_content, "lxml")
        movie_data = {}

        try:
            movie_data["genre"] = movie_page_soup.find_all("div",class_="thecontent clearfix")[0].find_all("p")[0].find_all("strong")[-1].text
            # Find and process the title element to check for [18+] marker
            title_element = movie_page_soup.find("h1", class_="title single-title entry-title")
            if title_element:
                title_text = title_element.text
                movie_data["adult"] = "yes" if "[18+]" in title_text else "no"
            else:
                movie_data["adult"] = np.nan

            # Extract additional information from the unordered list (ul)
            for list_item in movie_page_soup.find_all("ul", class_="wp-block-list")[0].find_all("li"):
                item_text = list_item.get_text().split(":")
                if len(item_text) == 2:
                    movie_data[item_text[0].strip()] = item_text[1].strip()

            print(f"Movie {movie_counter+1} complete")

        except Exception as error:
            # Catch and print any errors that occur during the data extraction
            print(f"An error occurred: {error}")

        # Increment the movie counter
        movie_counter += 1

        # Convert the movie data dictionary to a DataFrame and merge with final_data
        movie_series = pd.Series(movie_data).to_frame(name=movie_counter)
        final_data = final_data.merge(movie_series, left_index=True, right_index=True, how="outer")

    print(f"Page {page_number} complete")


Movie 1 complete
Movie 2 complete
Movie 3 complete
Movie 4 complete
Movie 5 complete
Movie 6 complete
Movie 7 complete
Movie 8 complete
Movie 9 complete
Movie 10 complete
Movie 11 complete
Movie 12 complete
Movie 13 complete
Movie 14 complete
Movie 15 complete
Movie 16 complete
Movie 17 complete
Movie 18 complete
Movie 19 complete
Page 1 complete
Movie 20 complete
Movie 21 complete
Movie 22 complete
Movie 23 complete
Movie 24 complete
Movie 25 complete
Movie 26 complete
Movie 27 complete
Movie 28 complete
Movie 29 complete
Movie 30 complete
Movie 31 complete
Movie 32 complete
Movie 33 complete
Movie 34 complete
Page 2 complete


In [5]:
final_data.to_csv("data/page_1_and_2.csv")